In [1]:
# необходимые зависимости

import numpy as np
import warnings
from binarytree import Node

In [2]:
warnings.filterwarnings('ignore', r'All-NaN slice encountered')

Вариант 8

A = np.array(
    [
        [np.nan, 4, 7, 9, 4],
        [3, np.nan, 1, 3, 2],
        [10, 9, np.nan, 1, 7],
        [4, 6, 1, np.nan, 9],
        [5, 2, 3, 7, np.nan]
    ],
    dtype=float
)

A

In [88]:
A = np.array(
    [
        [np.nan, 20, 18, 12, 8],
        [5, np.nan, 14, 7, 11],
        [12, 18, np.nan, 6, 11],
        [11, 17, 11, np.nan, 12],
        [5, 5, 5, 5, np.nan]
    ],
    dtype=float
)

A

array([[nan, 20., 18., 12.,  8.],
       [ 5., nan, 14.,  7., 11.],
       [12., 18., nan,  6., 11.],
       [11., 17., 11., nan, 12.],
       [ 5.,  5.,  5.,  5., nan]])

In [78]:
A = np.array(
    [
        [np.nan, 68, 73, 24, 70, 9],
        [58, np.nan, 16, 44, 11, 92],
        [63, 9, np.nan, 86, 13, 18],
        [17, 34, 76, np.nan, 52, 70],
        [60, 18, 3, 45, np.nan, 58],
        [16, 82, 11, 60, 48, np.nan]
    ],
    dtype=float
)

A

array([[nan, 68., 73., 24., 70.,  9.],
       [58., nan, 16., 44., 11., 92.],
       [63.,  9., nan, 86., 13., 18.],
       [17., 34., 76., nan, 52., 70.],
       [60., 18.,  3., 45., nan, 58.],
       [16., 82., 11., 60., 48., nan]])

In [171]:
class Node:
    
    def __init__(self, matrix: np.ndarray=None, weight: float=None):
        self.is_leaf = True
        self.left = None
        self.right = None
        self.matrix = matrix
        self.weight = weight
        
        
    def grow_left(self, matrix: np.ndarray, weight: float) -> None:
        self.left = Node(matrix, weight)
        self._check_growth()
        
        
    def grow_right(self, matrix: np.ndarray, weight: float) -> None:
        self.right = Node(matrix, weight)
        self._check_growth()
        
        
    def _check_growth(self) -> None:
        if self.left is not None and self.right is not None:
            self.is_leaf = False
    
    
    def find_least_leaf(self) -> Node:
        if self.is_leaf:
            return self
        elif self.left is not None and self.right is not None:
            left = self.left
            right = self.right
            
            if left.weight < right.weight:
                left.is_leaf = False
                return left
            else:
                right.is_leaf = False
                return right

In [205]:
class BranchAndBoundSolver:
    
    def __init__(self, initial_matrix: np.ndarray):
        self.initial_matrix = initial_matrix
        
        self.root = Node(
            *BranchAndBoundSolver.reduction(initial_matrix)
        )
        
        
    def solve(self):
        
#         while self.root.find_least_leaf().matrix.shape != (2, 2):
        for i in range(30):
            leaf = self.root.find_least_leaf()
            
            max_element_value, max_element_position = BranchAndBoundSolver.max_element_analysis(leaf.matrix)
            
            # ветвь решения с добавлением
            left_matrix = leaf.matrix.copy()
            i, j = max_element_position[0], max_element_position[1]
            
            left_matrix[i, j] = left_matrix[j, i] = np.nan
            left_matrix = np.delete(left_matrix, i, axis=0)
            left_matrix = np.delete(left_matrix, j, axis=1)
            
            left_matrix, penalty = BranchAndBoundSolver.reduction(left_matrix)
            
            leaf.grow_right(left_matrix.copy(), leaf.weight + penalty)
            
            
            # ветвь решения с пропусокм
            leaf.grow_left(leaf.matrix.copy(), leaf.weight + max_element_value)

    
    @staticmethod
    def max_element_analysis(matrix: np.ndarray):
        local_matrix = matrix.copy()
        new_matrix = np.zeros(shape=local_matrix.shape)
        
        for i in np.arange(local_matrix.shape[0]):
            for j in np.arange(local_matrix.shape[1]):
                stash = local_matrix[i, j]
                local_matrix[i, j] = np.nan
                
                axis0_min = np.nanmin(local_matrix[i])
                axis1_min = np.nanmin(local_matrix.T[j])
                
                new_matrix[i, j] = axis0_min + axis1_min
                
                local_matrix[i, j] = stash
                
        max_value = np.nanmax(new_matrix)
        
        max_value_position = np.array(
            np.where(new_matrix == max_value)
        ).T[0]
        
        return max_value, max_value_position
    
    
    @staticmethod
    def reduction(matrix: np.ndarray):
        local_matrix = matrix.copy()
        weight  = 0

        axis0_mins = np.nanmin(local_matrix, axis=1).reshape(-1, 1)
        weight  += axis0_mins.sum()
        local_matrix -= axis0_mins

        axis1_mins = np.nanmin(local_matrix, axis=0)
        weight  += axis1_mins.sum()
        local_matrix -= axis1_mins

        return local_matrix, weight    

In [206]:
solver = BranchAndBoundSolver(A)
solver.solve()

In [207]:
solver.root.right.matrix

array([[nan, 10.,  4.,  0.],
       [ 0.,  9.,  2., nan],
       [ 6., nan,  0.,  5.],
       [ 0.,  0., nan,  1.]])

In [208]:
BranchAndBoundSolver.max_element_analysis(solver.root.right.matrix)

(9.0, array([3, 1]))